In [14]:
import pandas as pd

In [15]:
# Load datasets
temp_data = pd.read_csv("combined_temperature.csv")
co2_data = pd.read_csv("annual-co2-emissions-per-country.csv")
credit_data = pd.read_csv("Credit_Ratings.csv")
finance_1 = pd.read_excel("Finance.xlsx")
finance_2 = pd.read_excel("Finance1.xlsx")
precip_data = pd.read_csv("API_AG.LND.PRCP.MM_DS2_en_csv_v2_23620.csv", skiprows=4)

In [16]:
# Process temperature
temp_avg = temp_data[(temp_data['Year'] >= 2011) & (temp_data['Year'] <= 2020)]
temp_avg = temp_avg.groupby('Code', as_index=False)['Annual Mean'].mean().rename(columns={'Annual Mean': 'temp_anomaly'})

In [17]:
# Process CO2
co2_avg = co2_data[(co2_data['Year'] >= 2011) & (co2_data['Year'] <= 2020)]
co2_avg = co2_avg.groupby('Code', as_index=False)['Annual CO₂ emissions'].mean().rename(columns={'Annual CO₂ emissions': 'co2_avg'})

In [18]:
# Process precipitation
precip_cols = [str(y) for y in range(2011, 2021)]
precip_avg = precip_data[['Country Code'] + precip_cols].copy()
precip_avg['precip_mm'] = precip_avg[precip_cols].mean(axis=1)
precip_avg = precip_avg[['Country Code', 'precip_mm']].rename(columns={'Country Code': 'Code'})

In [19]:
# Process credit ratings
credit_split = credit_data["Country;S&P;Moody's;DBRS;TE"].str.split(";", expand=True)
credit_split.columns = ['Country_Name', 'S&P', 'Moody', 'DBRS', 'TE']
credit_split = credit_split[['Country_Name', 'TE']].dropna()
credit_split['TE'] = pd.to_numeric(credit_split['TE'], errors='coerce')

In [20]:
# Process finance datasets
loan_access_row = finance_1[finance_1['Indicator Code'] == 'fin14']
finance_obstacle_row = finance_2[finance_2['Indicator Code'] == 'fin16']

loan_access = loan_access_row.drop(['Indicator', 'Indicator Code'], axis=1).T
finance_obstacle = finance_obstacle_row.drop(['Indicator', 'Indicator Code'], axis=1).T

loan_access.columns = ['loan_access']
finance_obstacle.columns = ['finance_obstacle']

loan_access = loan_access.reset_index().rename(columns={'index': 'country_year'})
finance_obstacle = finance_obstacle.reset_index().rename(columns={'index': 'country_year'})

loan_access[['Country', 'Year']] = loan_access['country_year'].str.extract(r'^(.*) \((\d{4})\)$')
finance_obstacle[['Country', 'Year']] = finance_obstacle['country_year'].str.extract(r'^(.*) \((\d{4})\)$')

loan_access = loan_access.dropna(subset=['Year'])
finance_obstacle = finance_obstacle.dropna(subset=['Year'])

loan_access['Year'] = loan_access['Year'].astype(int)
finance_obstacle['Year'] = finance_obstacle['Year'].astype(int)
loan_access['loan_access'] = pd.to_numeric(loan_access['loan_access'], errors='coerce')
finance_obstacle['finance_obstacle'] = pd.to_numeric(finance_obstacle['finance_obstacle'], errors='coerce')

finance_data = pd.merge(loan_access[['Country', 'Year', 'loan_access']],
                        finance_obstacle[['Country', 'Year', 'finance_obstacle']],
                        on=['Country', 'Year'], how='outer')

finance_data_latest = finance_data.sort_values('Year').drop_duplicates('Country', keep='last')

In [21]:
# Create ISO code mapping
country_code_map = temp_data[['Country', 'Code']].drop_duplicates().rename(columns={'Country': 'Country_Name'})
finance_credit = finance_data_latest.merge(credit_split, left_on='Country', right_on='Country_Name', how='inner')
finance_credit = finance_credit.merge(country_code_map, on='Country_Name', how='left')

In [22]:
# Merge all data sources
merged = finance_credit.merge(temp_avg, on='Code', how='inner')
merged = merged.merge(co2_avg, on='Code', how='inner')
merged = merged.merge(precip_avg, on='Code', how='inner')

In [23]:
# Final cleanup
merged = merged[['Country', 'loan_access', 'finance_obstacle', 'TE', 'temp_anomaly', 'co2_avg', 'precip_mm']]
merged = merged.rename(columns={
    'Country': 'country',
    'TE': 'credit_rating'
})


In [24]:
# Export CSV
merged.to_csv("merged_climate_finance_dataset.csv", index=False)

In [25]:
import pandas as pd

# Read the uploaded CSV file
df = pd.read_csv("merged_climate_finance_dataset.csv")

# Preview the data
df.head(100)

,country,loan_access,finance_obstacle,credit_rating,temp_anomaly,co2_avg,precip_mm
0,Brazil,59.2,45.2,46.0,25.710,496855247.0,1761.0
1,Fiji,37.8,6.7,35.0,24.825,1150481.7,2592.0
2,Gabon,9.0,30.4,23.0,25.380,5876826.9,1831.0
3,Chile,79.6,17.6,73.0,9.612,82378398.4,1522.0
4,Belize,43.9,66.8,20.0,25.988,569163.6,1705.0
...,...,...,...,...,...,...,...
95,Latvia,35.9,23.2,72.0,7.397,7406398.6,667.0
96,Turkmenistan,13.7,16.3,35.0,16.865,64068042.2,161.0
97,Tunisia,32.1,43.1,21.0,20.794,29618292.0,207.0
98,Uzbekistan,12.3,1.7,40.0,13.914,109841436.8,206.0
